In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
tqdm.pandas()
%matplotlib inline

In [ ]:
# df_name = 'us_covid_april_2021-april_2022_(2022-09-26).parquet'
# df_name = 'eur_covid_april_2021-april_2022_(2022-10-06).parquet'

df_name_1 = "eur_covid_april_2021-april_2022_(2022-10-06).parquet"
df_name_2 = "eur_covid_mar_2020-april_2021_(2022-10-04).parquet"

In [ ]:
df = pd.read_parquet(f'comp_files/analysis_{df_name}', 
                             columns=['article_id', 'date', 'publisher', 'title', 'page_num', 'source_type',
                               'city', 'author', 'LexileScore', 'prediction'])
print(f'Number of unique articles being analyzed: {df.article_id.nunique()}')

In [3]:
df1 = pd.read_parquet(f'comp_files/analysis_{df_name_1}', 
                             columns=['article_id', 'date', 'publisher', 'title', 'page_num', 'source_type',
                               'city', 'author', 'LexileScore', 'prediction'])
print(f'Number of unique articles being analyzed: {df1.article_id.nunique()}')

Number of unique articles being analyzed: 104330


In [4]:
df2 = pd.read_parquet(f'comp_files/analysis_{df_name_2}', 
                             columns=['article_id', 'date', 'publisher', 'title', 'page_num', 'source_type',
                               'city', 'author', 'LexileScore', 'prediction'])
print(f'Number of unique articles being analyzed: {df2.article_id.nunique()}')

Number of unique articles being analyzed: 185844


In [5]:
full_df = pd.concat([df1, df2], ignore_index=True)

In [6]:
full_df.head()

,article_id,date,publisher,title,page_num,source_type,city,author,LexileScore,prediction
0,2549006707.xml,2021-07-07,Guardian (Online) The,Ditching Covid restrictions now goes against a...,n/a,Web Resources,London (UK),None,1530,0
1,2549006707.xml,2021-07-07,Guardian (Online) The,Ditching Covid restrictions now goes against a...,n/a,Web Resources,London (UK),None,1530,0
2,2549006707.xml,2021-07-07,Guardian (Online) The,Ditching Covid restrictions now goes against a...,n/a,Web Resources,London (UK),None,1530,0
3,2549006707.xml,2021-07-07,Guardian (Online) The,Ditching Covid restrictions now goes against a...,n/a,Web Resources,London (UK),None,1530,0
4,2549006707.xml,2021-07-07,Guardian (Online) The,Ditching Covid restrictions now goes against a...,n/a,Web Resources,London (UK),None,1530,1


In [7]:
full_df['publisher'] = full_df['publisher'].astype('category')

In [8]:
full_df['article_id'] = pd.factorize(full_df['article_id'])[0]

In [9]:
full_df['article_id'] = full_df['article_id'].astype('int32')

In [10]:
full_df['prediction'] = full_df['prediction'].astype('int8')

In [11]:
full_df = full_df.drop(columns=['title', 'city', 'source_type', 'author', 'LexileScore'])

In [12]:
full_df['page_num'] = full_df['page_num'].astype('category')

In [13]:
full_df.to_parquet('email_analysis/eur_full_covid.parquet', index=False, compression='brotli')

In [ ]:
bruce = pd.DataFrame(full_df.publisher.value_counts()[:50])

In [ ]:
bruce['ideology'] = ''

In [ ]:
bruce.drop('publisher', axis=1, inplace=True)

In [ ]:
bruce.to_csv('email_analysis/us_bruce_publishers.csv')

In [ ]:
pd.read_csv('email_analysis/us_bruce_publishers.csv')

In [ ]:
editorial_df = pd.read_csv('csv/pre_explode_us_news_commentary.csv')

In [ ]:
compare_dfs = pd.read_csv('csv/analysis_us_news_apr2020-mar-2021.csv')

In [ ]:
compare_dfs.publisher.value_counts()

In [ ]:
np.sum(compare_dfs.article_id.isin(df['article_id'])) / len(compare_dfs)

In [ ]:
compare_dfs['article_id']

In [ ]:
df['article_id'].isin(editorial_df['article_id'])

In [ ]:
df.columns

In [ ]:
df['prediction'] = np.where(df['prediction'] == 0, -1, 1)

In [ ]:
first_groupby = {col_name:'first' for col_name in df.columns if col_name not in ['LexileScore', 'prediction']}
first_groupby['prediction'] = 'mean'
first_groupby['LexileScore'] = 'mean'

In [ ]:
# df = df.groupby('article_id').agg(first_groupby).groupby('date').agg(first_groupby)
article_grouped_df = df.groupby('article_id').agg(first_groupby)

In [ ]:
len(editorial_df)

In [ ]:
article_grouped_df['is_oped'] = np.where(article_grouped_df['article_id'].isin(editorial_df['article_id']), True, False)

In [ ]:
df = article_grouped_df.groupby('date').agg(first_groupby)

In [ ]:
df.head()

In [ ]:
df.city.value_counts()

In [ ]:
relevant_page_numbers = first_df.page_num.value_counts()[:10].index

In [ ]:
rel_page_df = first_df[first_df['page_num'].isin(relevant_page_numbers)].copy()

In [ ]:
rel_page_df.groupby('page_num')['prediction'].mean().sort_values()

In [ ]:
rel_page_df.groupby('page_num')['prediction'].mean().sort_values()

In [ ]:
rel_page_df['is_front_page'] = rel_page_df['page_num'].isin(['A.1', '1'])
first_df['is_front_page'] = first_df['page_num'].isin(['A.1', '1'])

In [ ]:
first_page_timeseries = rel_page_df[rel_page_df['is_front_page'] == True].groupby('date')['is_front_page'].count() / rel_page_df.groupby('date')['is_front_page'].count()
first_page_timeseries = first_df[first_df['is_front_page'] == True].groupby('date')['is_front_page'].count() / first_df[first_df['is_front_page'] == False].groupby('date')['is_front_page'].count()

In [ ]:
first_page_timeseries.rolling(5, win_type='gaussian',center=False,).mean(std=2).plot()

We want to get the relative number of front page articles being published compared overtime since all publication numbers went down during the end of omicron wave.

In [ ]:
# front_page_df.groupby(['date', 'page_num']).count()['article_id'].plot()

In [ ]:
df.prediction.plot()

In [ ]:
df = df.set_index(['date', 'article_id']).groupby(level=0).mean()

In [ ]:
df['prediction'].mean()

In [ ]:
grouped_df = predict_df.groupby(['article_id']).mean()

In [ ]:
date_row = predict_df.groupby(['article_id'])['date'].first()

In [ ]:
new = pd.concat([grouped_df, date_row], axis=1)

In [ ]:
new.groupby('date').mean()['prediction'].min()

In [ ]:
np.sum(predict_df.groupby('article_id')['prediction'].mean() > 0) / predict_df.article_id.nunique()

In [ ]:
np.sum(predict_df.groupby('article_id')['prediction'].mean() > 0) / predict_df.article_id.nunique()

**Sentiment based on Publisher and Number of articles published**

**Sentiment by Page Number**

In [ ]:
page_counts = predict_df.page_num.value_counts()[:50]
page_nums = page_counts.index
sum_pages = predict_df.groupby('page_num')['prediction'].sum().loc[page_nums]
page_num_df = pd.concat((page_counts,sum_pages),axis=1).reset_index(level=0)
page_num_df.rename({'index':'page', 'prediction':'page_prediction', 'page_num': 'total_page'}, axis=1, inplace=True)

**Sentiment by publisher**

In [ ]:
publisher_counts = predict_df.publisher.value_counts()[:50]
publisher_name = publisher_counts.index
sum_publisher = predict_df.groupby('publisher')['prediction'].sum().loc[publisher_name]
publisher_df = pd.concat((publisher_counts,sum_publisher),axis=1).reset_index(level=0)
publisher_df.rename({'index':'publisher_name', 'publisher': 'total_published', 'prediction':'publisher_prediction'}, axis=1, inplace=True)

In [ ]:
publisher_page_df = pd.concat((publisher_df, page_num_df),axis=1)
publisher_page_df.to_parquet('email_analysis/publisher_page_' + df_name)

**Number of publications per day**

In [ ]:
num_articles_per_day = predict_df.groupby('date').nunique()['article_id'].reset_index(level=0)
num_articles_per_day = num_articles_per_day.rename({'article_id':'articles_per_day'},axis=1)

In [ ]:
predict_df.prediction.mean()

In [ ]:
df = predict_df.merge(num_articles_per_day, on='date', how='outer')

In [ ]:
df.date.max()

In [ ]:
df = df.groupby('date', as_index=False)[['date','prediction', 'articles_per_day']].mean()

In [ ]:
# # ONLY FOR NEWS TITLES
# df = predict_df.groupby('date', as_index=False).mean()
# df = df[['date', 'prediction']]

In [ ]:
df.head(10)

In [ ]:
save_file = 'email_analysis/' + df_name_csv
df.to_csv(save_file, index=False)

In [ ]:
save_file

In [ ]:
predict_df.date.max()